<a href="https://colab.research.google.com/github/JSJeong-me/Uniinfo-Machine-Vision/blob/main/U0805/kaggle_cv_lec_06_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 소개  
이제 합성곱 분류자의 기본 요소들을 다 알았으니, 더 심화된 주제로 넘어가자.  
  
  이번 포스트에서는, 이미지 분류자의 성능을 더 높여줄 하나의 트릭을 배울 것이다 : 이는 데이터 증대(Data Augmentation)이다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp /content/drive/MyDrive/archive.zip .

In [3]:
!cp /content/drive/MyDrive/archive-model.zip .

# 가짜 데이터의 유용성  
머신 러닝 모델의 성능을 향상시키는 가장 좋은 방법은 더 많은 데이터로 학습시키는 것이다. 모델이 '배울 수 있는' 데이터가 더 많을수록, 특정 이미지에서 중요한 요소를 더 효과적으로 구분해낼 수 있다.  
  
  데이터를 더 얻을 수 있는 좋은 방법은 이미 가지고 있는 데이터를 사용하는 것이다. 데이터셋의 이미지를 클래스를 보존하게끔 변형한다면, 분류자가 이러한 '변형'을 무시하도록 훈련시킬 수 있다. 예를 들어, 이미지 상 차가 왼쪽을 향하는지 오른쪽을 향하는지는 그것이 트럭이 아니라 차라는 사실에 영향을 미치지 않는다. 따라서 훈련 데이터를 뒤집어 데이터를 증대(augment)한다면, 분류자는 '왼쪽 또는 오른쪽'은 무시해야 할 특성임을 학습할 것이다.  
    
  이것이 데이터 증대의 기본적인 아이디어이다 : 본래 데이터와 비슷한 가짜 데이터를 추가하여 분류자의 성능 향상을 꾀한다.

In [4]:
!pwd

/content


In [ ]:
!unzip ./archive.zip

Archive:  ./archive.zip
replace train/Car/00002.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip ./archive-model.zip

# 데이터 증대 사용하기  
데이터셋을 증대하는데에는 많은 변형 기법이 사용된다. 이미지를 회전하거나, 색 또는 대비를 변형하거나, 이미지를 왜곡하는 등 조합하는 과정에서 발생하는 많은 기법이 존재한다. 아래는 하나의 이미지가 변형될 수 있는 예시를 보여준다.  
<figure>
<img src="https://i.imgur.com/UaOm0ms.png" width=400, alt="Sixteen transformations of a single image of a car.">
</figure>  
데이터 증대는 대부분 '온라인'으로 이루어진다 - 즉, 데이터가 훈련을 위해 신경망으로 입력되면서 이루어진다. 훈련은 대부분 데이터의 미니 배치 단위로 이루어짐을 떠올려보자. 아래는 데이터 증대가 사용될 때 16개의 배치 이미지를 시각화한다.  
<figure>
<img src="https://i.imgur.com/MFviYoE.png" width=400, alt="A batch of 16 images with various random transformations applied.">
</figure>  
훈련 과정에서 이미지가 사용될때마다, 새로운 랜덤한 변형이 적용된다. 덕분에 모델은 항상 새로운 이미지를 볼 수 있다. 이런 훈련 데이터에 대한 추가적인 변형이 새로운 데이터 형성에 도움이 된다.  
  
  그런데 모델로 해결해야 할 문제를 위해서 모든 변형이 도움이 되는 것은 아니다. 더욱이, 변형이 이루어질 때 클래스가 섞이면 안된다. 예를 들어 숫자 분류기를 훈련시킬 때 이미지를 회전한다면 6과 9가 섞일 수 있다. 결국 가장 효과적인 데이터 증대 방법을 찾는 것은 다른 머신 러닝 문제 해결 방법과 동일하다. 일단 해보는 것이다.

# 예시 - 데이터 증대와 훈련  
케라스는 데이터 증대의 두 가지 방법을 제시한다. 첫 번째 방법은 데이터 파이프라인에 ImageDataGenerator 등의 함수를 포함하는 것이다. 두 번째 방법은 케라스의 preprocessing 레이어를 사용하여 모델을 정의할 때 포함하는 것이다. 아래 예시에서는 후자의 방법을 택한다.   
  
  이 예시를 통해 데이터 증대를 사용하여 분류자의 성능을 어떻게 향상시킬 수 있는지를 배울 것이다.

In [1]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# 랜덤 시드
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# matplotlib 사전 설정
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large', 
      titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings('ignore') # 경고 메시지 무시해서 출력셀 정리

# 훈련, 검증 데이터 불러오기
ds_train_ = image_dataset_from_directory(
    './train',
    labels = 'inferred',
    label_mode = 'binary',
    image_size = [128,128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = True
)
ds_valid_ = image_dataset_from_directory(
    './valid',
    labels = 'inferred',
    label_mode = 'binary',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = False
)

# 데이터 파이프라인
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (ds_train_.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))
ds_valid = (ds_valid_.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


## Step 2 - 모델 정의하기  
데이터 증대의 효과를 보여주기 위해, 이전에 사용한 모델의 코드를 조금 변형할 것이다.

In [2]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
# TF 2.2의 새로운 특성이다
import tensorflow.keras.layers.experimental.preprocessing as preprocessing

pretrained_base = tf.keras.models.load_model(
    './cv-course-models/vgg16-pretrained-base'
)
pretrained_base.trainable = False

model = keras.Sequential([
    # 전처리
    preprocessing.RandomFlip('horizontal'), # 좌우 반전
    preprocessing.RandomContrast(0.5), # 대비도 50% 변형
    # 베이스
    pretrained_base,
    # 헤드
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

## Step 3 - 훈련과 결과 분석  

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

history = model.fit(
    ds_train,
    validation_data = ds_valid,
    epochs = 1 # 30
)

80/80 [==============================] - ETA: 0s - loss: 0.4963 - binary_accuracy: 0.7604 

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy','val_binary_accuracy']].plot();

첫 번째 튜토리얼에서의 훈련과 검증 곡선은 빠르게 수렴했고, 이는 정규화의 영향을 받았다는 것을 의미한다. 이번 모델의 학습 곡선은 서로 가까운 거리에 위치하고, 검증 오차와 정확도에 대해 향상된 성능을 보인다. 따라서 데이터셋 증대가 도움이 되었다는 것을 확인할 수 있다.